# tensorflow in kerasを利用した手書き文字分類

ニューラルネットワークを構築する為のtensorflowラッパーライブラリkerasを利用した実装
tensorflow in kerasを使う事で、簡単にニューラルネットワークを書きつつ、
tensorflowのtensorboardやsession、最適化された計算を利用できます。

pre release版tensorflow v1.1.0rc の contrib(将来マージされるかもしれない機能)を利用しています。


### インポート

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from tensorflow.contrib.keras.python import keras
from tensorflow.contrib.keras.python.keras import backend as K

### データセット

In [2]:
def load_data():
    dirn = './MNIST_data'
    mnist = input_data.read_data_sets(dirn, one_hot=True)

    print(mnist.train.num_examples, 'train samples')
    print(mnist.test.num_examples, 'test samples')
    print(mnist.validation.num_examples, 'validation samples (not used)')

    return mnist

### モデルの作成



In [3]:
def mlp_model(input):
    # MLP network model
    with tf.variable_scope('mlp_model'):
        x = keras.layers.Dense(units=512, activation='relu')(input)
        x = keras.layers.Dropout(0.2)(x)
        x = keras.layers.Dense(units=512, activation='relu')(x)
        x = keras.layers.Dropout(0.2)(x)
        y_pred = keras.layers.Dense(units=10, activation='softmax')(x)

    return y_pred

### 実行



In [4]:
mnist = load_data()
# tensorflow placeholders
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
# define TF graph
y_pred = mlp_model(x)
loss = tf.losses.softmax_cross_entropy(y_, y_pred)
train_step = tf.train.AdagradOptimizer(0.05).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print('Training...')
    for i in range(10001):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        train_fd = {x: batch_xs, y_: batch_ys, K.learning_phase(): 1}
        train_step.run(feed_dict=train_fd)
        if i % 1000 == 0:
            batch_xv, batch_yv = mnist.test.next_batch(200)
            val_accuracy = accuracy.eval(
                {x: batch_xv, y_: batch_yv, K.learning_phase(): 0})
            print('  step, accurary = %6d: %6.3f' % (i, val_accuracy))

    test_fd = {x: mnist.test.images, y_: mnist.test.labels,
                K.learning_phase(): 0}
    test_accuracy = accuracy.eval(feed_dict=test_fd)
    print('Test accuracy:', test_accuracy)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
55000 train samples
10000 test samples
5000 validation samples (not used)
Training...
  step, accurary =      0:  0.075
  step, accurary =   1000:  0.915
  step, accurary =   2000:  0.935
  step, accurary =   3000:  0.965
  step, accurary =   4000:  0.960
  step, accurary =   5000:  0.960
  step, accurary =   6000:  0.985
  step, accurary =   7000:  0.970
  step, accurary =   8000:  0.990
  step, accurary =   9000:  0.960
  step, accurary =  10000:  0.970
Test accuracy: 0.9749
